In [ ]:
#%matplotlib notebook
%matplotlib inline
#%config InlineBackend.figure_formats=['svg']
#%config InlineBackend.figure_formats=['pdf']

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

from matplotlib.colors import LogNorm
from matplotlib.ticker import FuncFormatter

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

## NPE

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
#WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0       # 0 for gamma, 1 for protons
MIN_NPE = 0    # cut on SUM_PE_CLEANED_IMAGE

XLINE = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

In [ ]:
#full_df.groupby(["Type","Part","Success"]).count()

### NPE on cleaned image

In [ ]:
fig, ax = plt.subplots()

df1 = df[df.peSum_tc > 0]['peSum_tc']
df2 = df['peSum']

df1.plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="blue",
              label="Tailcut ({})".format(len(df1)),  legend=True)
df2.plot.hist(alpha=0.2, bottom=0.1, bins=np.logspace(0.1, 6., 90), color="green",
              label="Reference ({})".format(len(df2)), legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

In [ ]:
fig, ax = plt.subplots()

df1 = df[df.peSum_wt > 0]['peSum_wt']
df2 = df['peSum']

df1.plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="red",
              label="Wavelets ({})".format(len(df1)),  legend=True)
df2.plot.hist(alpha=0.2, bottom=0.1, bins=np.logspace(0.1, 6., 90), color="green",
              label="Reference ({})".format(len(df2)), legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

In [ ]:
fig, ax = plt.subplots()

df1 = df[df.peSum_tc > 0]['peSum_tc']
df2 = df[df.peSum_wt > 0]['peSum_wt']

df1.plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="blue",
              label="Tailcut ({})".format(len(df1)),  legend=True)
df2.plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="red",
              label="Wavelets ({})".format(len(df2)),  legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

## Ratio

In [ ]:
def plot_ratio(ax, res_tuple_tc, res_tuple_wt):
    val_of_bins_tc, bins_tc, patches_tc = res_tuple_tc
    val_of_bins_wt, bins_wt, patches_wt = res_tuple_wt
    edges_of_bins = bins_tc

    # Set ratio where val_of_bins_data is not zero
    ratio = np.divide(val_of_bins_wt,
                      val_of_bins_tc,
                      where=(val_of_bins_tc != 0))

    # Compute error on ratio (null if cannot be computed)
    # This is wrong as it's made for Gaussian distributions and here we have Poisson distribution
    error = np.divide(val_of_bins_wt * np.sqrt(val_of_bins_tc) + val_of_bins_tc * np.sqrt(val_of_bins_wt),
                       np.power(val_of_bins_tc, 2),
                       where=(val_of_bins_tc != 0))

    ax.set_ylabel('Ratio (WT/TC)', fontsize=20)
    ax.axhline(y=1, linewidth=2, linestyle='--', color='gray', alpha=0.5)

    bincenter = 0.5 * (edges_of_bins[1:] + edges_of_bins[:-1])
    ax.errorbar(bincenter, ratio, yerr=error, fmt='o', color='k', elinewidth=3, capsize=4, capthick=3, linewidth=6)
    ax.plot(bincenter, ratio, 'ok', linewidth=6)
    
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 9))

# AX1

#wt['peSum'].plot.hist(bottom=0.1, legend=True)
BINS = np.logspace(0.1, 6., 30)

df1 = df[df.peSum_tc > 0]['peSum_tc']
df2 = df[df.peSum_wt > 0]['peSum_wt']

res_tuple_tc1 = ax1.hist(df1,
                         label="Tailcut ({})".format(len(df1)),
                         bins=BINS,
                         linewidth=2, alpha=0.5, color='blue',  histtype="step")

res_tuple_wt1 = ax1.hist(df2,
                         label="Wavelets ({})".format(len(df2)),
                         bins=BINS,
                         linewidth=2, alpha=0.5, color='red', histtype="step")

ax1.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax1.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax1.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax1.set_xlabel("Number of photoelectrons", fontsize=20)
ax1.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax1.set_xscale('log')

ax1.legend(prop={'size': 18})

# AX2

plot_ratio(ax2, res_tuple_tc1, res_tuple_wt1)
ax2.set_xscale('log')
ax2.set_title("WT/TC")

ax2.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax2.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax2.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax2.set_xlabel("Number of photoelectrons", fontsize=20)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['peSum'], df['peSum_tc'], '.b', alpha=0.1, label="tc")
ax.plot(df['peSum'], df['peSum_wt'], '.r', alpha=0.1, label="wt")

ax.plot([0,100000], [0,100000], '--k', alpha=0.5, linewidth=2)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("npe ref", fontsize=20)
ax.set_ylabel("npe clean", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = df['peSum'], df['peSum_tc']
x2, y2 = df['peSum'], df['peSum_wt']

xbins = np.logspace(1, 4, 30)
ybins = np.logspace(1, 4, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.1*max_value, 0.3*max_value, 0.6*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.set_xlim(1e1, 1e4)
ax.set_ylim(1e1, 1e4)

ax.plot([0,100000], [0,100000], '--k', alpha=0.5, linewidth=2)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("npe ref", fontsize=20)
ax.set_ylabel("npe clean", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ['TC','WT']
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

## PE Sum difference per image

In [ ]:
df['peSum_wt-tc'] = df['peSum_wt'] - df['peSum_tc']

dfn = df[df["peSum_wt-tc"] < 0]
dfp = df[df["peSum_wt-tc"] >= 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

ax1.plot(np.abs(dfn["peSum_wt-tc"]), dfn['peSum'], ".r",
         alpha=0.1, label="WT - TC (per image) < 0 ({})".format(len(dfn)))
ax2.plot(dfp["peSum_wt-tc"], dfp['peSum'], ".r",
         alpha=0.1, label="WT - TC (per image) > 0 ({})".format(len(dfp)))

fig.suptitle("WT - TC (per image)", fontsize=20)

ax1.set_yscale('log')
ax1.set_xscale('log')

ax2.set_yscale('log')
ax2.set_xscale('log')

ax1.set_xlabel("Sum PE (negative differences)", fontsize=20)
ax1.set_ylabel("Sum PE ref.", fontsize=20)

ax2.set_xlabel("Sum PE (positive differences)", fontsize=20)
ax2.set_ylabel("Sum PE ref.", fontsize=20)

ax1.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)
ax2.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

x1 = np.abs(dfn["peSum_wt-tc"])
x2 = dfp["peSum_wt-tc"]
y1 = dfn['peSum']
y2 = dfp['peSum']

xmin1 = np.log10(x1.min())
xmax1 = np.log10(x1.max())
ymin1 = np.log10(y1.min())
ymax1 = np.log10(y1.max())

xmin2 = np.log10(x2.min())
xmax2 = np.log10(x2.max())
ymin2 = np.log10(y2.min())
ymax2 = np.log10(y2.max())

xbins1 = np.logspace(xmin1, xmax1, 70) # <- make a range from 10**xmin to 10**xmax
ybins1 = np.logspace(ymin1, ymax1, 70) # <- make a range from 10**ymin to 10**ymax

xbins2 = np.logspace(xmin2, xmax2, 70) # <- make a range from 10**xmin to 10**xmax
ybins2 = np.logspace(ymin2, ymax2, 70) # <- make a range from 10**ymin to 10**ymax

counts1, _, _ = np.histogram2d(x1, y1, bins=(xbins1, ybins1))
pcm1 = ax1.pcolormesh(xbins1, ybins1, counts1.T, norm=LogNorm())
fig.colorbar(pcm1, ax=ax1)

counts2, _, _ = np.histogram2d(x2, y2, bins=(xbins2, ybins2))
pcm2 = ax2.pcolormesh(xbins2, ybins2, counts2.T, norm=LogNorm())
fig.colorbar(pcm2, ax=ax2)

#im1 = ax1.hexbin(x1, y1, gridsize=40)
#im2 = ax2.hexbin(x2, y2, gridsize=40)

#fig.colorbar(im1, ax=ax1)
#fig.colorbar(im2, ax=ax2)

fig.suptitle("WT - TC (per image)", fontsize=20)

ax2.set_xlim(0.0001, 1000)
ax2.set_ylim(1, 10000)

ax1.set_title("WT - TC (per image) < 0 ({})".format(len(dfn)))
ax2.set_title("WT - TC (per image) > 0 ({})".format(len(dfp)))

ax1.set_yscale('log')
ax1.set_xscale('log')

ax2.set_yscale('log')
ax2.set_xscale('log')

ax1.set_xlabel("Sum PE (negative differences)", fontsize=20)
ax1.set_ylabel("Sum PE ref.", fontsize=20)

ax2.set_xlabel("Sum PE (positive differences)", fontsize=20)
ax2.set_ylabel("Sum PE ref.", fontsize=20)

ax1.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)
ax2.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

## PE sum TC VS PE sum WT

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

ax.plot(df['peSum_wt'], df['peSum_tc'], ".r", alpha=0.1)

ax.plot([0,100000], [0,100000], '--k', alpha=0.5, linewidth=2)

ax.set_title("PE sum TC VS PE sum WT", fontsize=20)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(1, 100000)
ax.set_ylim(10, 100000)

ax.set_xlabel("Sum PE WT", fontsize=20)
ax.set_ylabel("Sum PE TC", fontsize=20)

In [ ]:
# %load /Users/jdecock/git/pub/jdhp/snippets/python/matplotlib/hist2d_hexa_logscale_xy.py

fig, ax = plt.subplots(nrows=1, ncols=1)

x = np.log10(df['peSum_wt'])
y = np.log10(df['peSum_tc'])

im = ax.hexbin(x, y, gridsize=100) #, norm=LogNorm())
fig.colorbar(im, ax=ax)

# Use "10^n" instead "n" as ticks label
func_formatter = lambda x, pos: r'$10^{{{}}}$'.format(int(x))
ax.xaxis.set_major_formatter(FuncFormatter(func_formatter))
ax.yaxis.set_major_formatter(FuncFormatter(func_formatter))

ax.plot([0,5], [0,5], '--w', alpha=0.5, linewidth=3)

ax.set_xlim(0, 4.5)
ax.set_ylim(1, 4.5)

ax.set_title("PE sum TC VS PE sum WT", fontsize=20)

ax.set_xlabel("Sum PE WT", fontsize=28)
ax.set_ylabel("Sum PE TC", fontsize=28)

In [ ]:
# %load /Users/jdecock/git/pub/jdhp/snippets/python/matplotlib/hist2d_scatter_plot_logscale_xy.py

fig, ax = plt.subplots()

x = np.log10(df['peSum_wt'])
y = np.log10(df['peSum_tc'])

xbins = np.linspace(x.min(), x.max(), 150)
ybins = np.linspace(y.min(), y.max(), 150)

hist, xedges, yedges = np.histogram2d(x, y, bins=(xbins, ybins))

# See http://stackoverflow.com/questions/27156381/python-creating-a-2d-histogram-from-a-numpy-matrix

xidx = np.clip(np.digitize(x, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(y, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]

sc = ax.scatter(x, y,
                 c=c,
                 s=5,
                 marker='o',
                 #cmap='gnuplot2',
                 linewidth=0,
                 alpha=1)

fig.colorbar(sc, ax=ax)

# Use "10^n" instead "n" as ticks label
func_formatter = lambda x, pos: r'$10^{{{}}}$'.format(int(x))
ax.xaxis.set_major_formatter(FuncFormatter(func_formatter))
ax.yaxis.set_major_formatter(FuncFormatter(func_formatter))

ax.plot([0,5], [0,5], '--k', alpha=0.5, linewidth=3)

ax.set_xlim(0, 5)
ax.set_ylim(1, 5)
#ax.set_xlim(1.5, 2.5)
#ax.set_ylim(1.5, 2.5)

ax.set_title("PE sum TC VS PE sum WT", fontsize=20)

ax.set_xlabel("Sum PE WT", fontsize=28)
ax.set_ylabel("Sum PE TC", fontsize=28)